<center>

# فاز دوم پروژه




<div dir="rtl">
     در صورت هرگونه سوال به ایمیل‌های زیر پیام داده یا از طریق سایر پلتفرم‌ها(در تلگرام از
    alirezadaqiq یا sina_tav ) پیگیری کنید
    
[Sina Tavakkoli](mailto:stavakkoli1999@gmail.com)
     <br>
[Alireza Daqiq](mailto:alireza.daghigh1999@gmail.com)
    
 </div>

<div dir="rtl">

## I. خلاصه انجام پروژه

در این تمرین, می‌خواهیم یک تصحیح‌کننده‌ی غلط‌املایی احتمالاتی بسازیم که غلط‌های موجود در کوئری را اصلاح کند.
   ورودی ما در این مسئله, کوئری $R$ است و هدف پیدا کردن کوئری درست $Q$ است که احتمال $P(Q\mid R)$ را بیشینه کند. 
    بر اساس قضیه‌ی بیز داریم :
$$
    P(Q\mid R) = \frac{P(R\mid Q)P(Q)}{P(R)}\propto P(R\mid Q)P(Q).
$$
از آنجایی که هدف ما پیدا کردن $Q$یی است که عبارت $P(Q\mid R)$ را بیشینه کند, تنها کافی‌ست که $P(R\mid Q)P(Q)$ را بیشینه کنیم.
    حال با توجه به چیزهایی که گفته شد, مصحح غلط‌املایی احتمالاتی ما باید چهار بخش داشته باشد:
  1.  **مدل ‌زبانی.**(Language Model) \
      توزیع اولیه‌ی یونیگرام‌ها و بایگرام‌ها را حساب می‌کند. این کار به ما اجازه می‌دهد که $P(Q)$ حساب کنیم. ما از maximum-likelihood estimation استفاده می‌کنیم, که وقوع توکن یونیگرام و بایگرام‌ها را در training corpus برای تعیین کردن احتمالات اولیه می‌شمرد.
  2. **مدل احتمال ویرایش.** (Edit Probability Model)\
      احتمال خطاهایی که ممکن است در یک کوئری اتفاق بیافتد را حساب می‌کند. این به ما اجازه می‌دهد که $P(R\mid Q)$ را محاسبه کنیم. به طور خاص، این کامپوننت احتمال کرکترهایی را حساب می‌کند که در یک کوئری ترم به اشتباه حذف، درج، جایگزین یا ترنسپوز شده‌اند.
  3. **تولیدکننده‌ی نامزدها.** (Candidate Generator)\
      کوئری خام $R$ که توسط کاربر ثبت شده است را می‌گیرد و نامزد‌های $Q$ را تولید می‌کند.
  4. **امتیازدهنده‌ی نامزدها.** (Candidate Scorer)\
      موارد (1), (2) و (3) را برای محاسبه‌ی $Q^{*} = \arg\max_{Q}P(Q\mid R)$ ترکیب می‌کند. یعنی برای هر $Q$ که توسط تولید‌کننده‌ی نامزدها تولید شده است، امتیازدهنده از مدل‌زبان برای محاسبه‌ی $P(Q)$ و از مدل احتمال ویرایش برای محاسبه‌ی $P(R\mid Q)$ استفاده می‌کند و در نهایت $Q$ را انتخاب می‌کند که مقدار $P(Q)P(R\mid Q)$ بیشینه می‌کند.
</div>

<div dir="rtl">

## II. جزئیات پروژه


  1. [تسک اول: تصحیح املا با استفاده از Uniform Edit Costs](#uniform)
  2. [تسک دوم: تصحیح املا با استفاده از Empirical Edit Costs](#empirical)
  3. [گزارش مکتوب](#written)
    </div>

<a id="dataset"></a>

<div dir="rtl">

## III. دیتاست


برای این تمرین, ما برای داده‌های تمرین و تست  از یکی از دیتا‌ست‌های دانشگاه استنفورد با اندکی تغییرات استفاده کردیم
برای متون هم ازیکی از آزمایشگاه‌های این دانشگاه بهره بردیم.

   <div dir="rtl">
   دیتاست از این <a href="https://drive.google.com/file/d/17ukKiVAYtbBuL_zI8_2IhZ8uzxkMJdPs/view?usp=sharing"> لینک </a> قابل دسترسی است. آن‌را دانلود کرده و کنار notebook قرار دهید و سلول زیر را اجرا کنید . دیتاست شامل سه بخش زیر است:
    </div>
    <ul>
       <li>  corpusها : در پوشه مربوط به آن ۱۰ فایل می‌بینید که هر خط هر فایل از یک مستند است. از tokenهای این بخش در ساخت model استفاده کنید</li>
       <li> training set: شامل جفت‌هایی از عبارات غلط و درست‌ آنها که با حداکثر یک عدد که edit distanceآنهاست جدا شده‌اند</li>
       <li>  dev set : در پوشه مربوطه سه فایل وجود دارد, ورژن عبارات با غلط املایی, ورژن صحیح آنها در gold.txt  و ورژن تصحیح شده توسط google در google.txt </li>
    </ul>    
    </div>

In [3]:

# Import modules
import math
import os
import urllib.request
import zipfile
from collections import Counter
from tqdm import tqdm

In [4]:
# Download dataset
# go to this url and download dataset https://drive.google.com/file/d/17ukKiVAYtbBuL_zI8_2IhZ8uzxkMJdPs/view?usp=sharing
data_dir = 'MIR2-data'

# Unzip dataset
with zipfile.ZipFile('{}.zip'.format(data_dir), 'r') as zip_fh:
    zip_fh.extractall()
print('Data unzipped to {}...\n'.format(data_dir))

# Print the directory structure
print('Directory Structure:')
print(data_dir + os.path.sep)
for sub_dir in os.listdir(data_dir):
    if not sub_dir.startswith('.'):
        print('  - ' + sub_dir + os.path.sep)

Data unzipped to MIR2-data...

Directory Structure:
MIR2-data/
  - dev_set/
  - training_set/
  - corpus/


<div dir='rtl'>
در تغییر ساختار کد در هر بخش آزاد هستید, فقط خروجی‌ها باید مورد انتظار بوده و تغییر داده شده در گزارش مکتوب آورده شود


<div dir="rtl">

<a id='uniform'></a>
## IV. تسک اول:

### IV.1. مدل‌زبان

  حال می‌خواهیم یک مدل‌زبان برا پیشبینی $P(Q)$ از training corpusها بسازیم. رفتار ما با $P(Q)$ به صورت مجموعه‌ای از ترم‌های $(w_1, \ldots, w_n)$ است که:
  $$
P(w_1, \ldots, w_n) = P(w_1)P(w_2\mid w_1)\cdots P(w_n\mid w_{n-1}),
$$
  که $P(w_1)$ در واقع احتمال یونیگرام ترم$w_1$  و $P(w_{i}\mid w_{i-1})$ و احتمال بایگرام $(w_{i-1}, w_i)$ برای $i \in \{2, \ldots, n\}$ است.
</div>

<div  style="direction:rtl" >

مدل‌زبان برای به دست آوردن احتمالات از MLE استفاده می کند:
$$
\begin{align*}
  P_{\text{MLE}}(w_i) & = \frac{\texttt{count}(w_i)}{T},
  &
  P_{\text{MLE}}(w_i\mid w_{i-1}) & = \frac{\texttt{count}((w_{i}, w_{i-1}))}{\texttt{count}(w_{i-1})},
\end{align*}
$$
که $T$ تعداد کل توکن‌ها در corpus است و $\texttt{count}$ تعداد یونیگرام‌ها و بایگرام‌هاست.

برای به‌دست آوردن count برای یونیگرام‌ها و بایگرام‌ها در corpus، کد زیر را کامل کنید:
    
</div>

In [5]:

class LanguageModel:
    """Models prior probability of unigrams and bigrams."""

    def __init__(self, corpus_dir='MIR2-data/corpus', lambda_=0.1):
        """Iterates over all whitespace-separated tokens in each file in
        `corpus_dir`, and counts the number of occurrences of each unigram and
        bigram. Also keeps track of the total number of tokens in the corpus.

        Args:
            corpus_dir (str): Path to directory containing corpus.
            lambda_ (float): Interpolation factor for smoothing by unigram-bigram
                interpolation. You only need to save `lambda_` as an attribute for now, and
                it will be used later in `LanguageModel.get_bigram_logp`. See Section
                IV.1.2. below for further explanation.
        """
        self.lambda_ = lambda_
        self.total_num_tokens = 0        # Counts total number of tokens in the corpus
        self.unigram_counts = Counter()  # Maps strings w_1 -> count(w_1)
        self.bigram_counts = Counter()   # Maps tuples (w_1, w_2) -> count((w_1, w_2))

        ### Begin your code

        ### End your code

<div dir="rtl">

   حال که تعداد یونیگرام‌ها و بایگرام‌ها را محاسبه کردیم، باید احتمال کوئری‌ها را نیز بسنجیم. قبل از ان به بایگرام‌هایی بپردازیم که هیچ‌گاه در corpus ندیدیم.

</div>

<div dir="rtl">

#### IV.1.2.  هموارسازی با استفاده از درون‌یابی ( Smoothing by Interpolation)

مدل احتمال یونیگرام به عنوان مجموعه لغات ما هم خواهد بود. چون فرض می‌کنیم که کوئری ما از مستند corpus آمده است، درنتیجه در احتمالات یونیگرام خود نیازی به [هموارسازی لاپلاس](https://en.wikipedia.org/wiki/Additive_smoothing) نداریم، چون نامزدهای ما از همین لغت‌نامه آمده‌اند. اما حتی اگر دو کوئری ترم داشتیم که هردوی آن‌ها جزو کوئری زبان ما بودند، تضمینی نیست که بایگرام متناظر آن‌ها در training corpus وجود داشته باشند. برای مدیریت این پراکندگی داده ما احتمالات یونیگرام و بایگرام را برای به دست آوردن احتمال شرطی نهایی خود *درون‌یابی* می‌کنیم:
$$
P(w_2\mid w_1) = \lambda P_{\text{MLE}}(w_2) + (1 - \lambda)P_{\text{MLE}}(w_2\mid w_1).
$$
با قرار دادن یک مقدار کوچک به جای $\lambda$ (برای مثال، 0.1) شروع می‌کنیم و سپس با تغییر این متغیر آزمون و خطا می‌کنیم تا دقت تصحیح ما در دیتاست توسعه بالاتر برود. توجه کنید که در این مورد برای دیتاست مورد نظر overﬁt نکنید. می‌توانید یک قسمت کوچک از داده‌های خود را برای tune کردن پارامترها نگه‌دارید.

توابع زیر را برای کامل کردن کلاس `LanguageModel` بنویسید.

In [6]:
# NOTE: Syntax on the following line just extends the `LanguageModel` class
class LanguageModel(LanguageModel):
    def get_unigram_logp(self, unigram):
        """Computes the log-probability of `unigram` under this `LanguageModel`.

        Args:
            unigram (str): Unigram for which to compute the log-probability.

        Returns:
            log_p (float): Log-probability of `unigram` under this
                `LanguageModel`.
        """
        ### Begin your code

        ### End your code

    def get_bigram_logp(self, w_1, w_2):
        """Computes the log-probability of `unigram` under this `LanguageModel`.

        Note:
            Use self.lambda_ for the unigram-bigram interpolation factor.

        Args:
            w_1 (str): First word in bigram.
            w_2 (str): Second word in bigram.

        Returns:
            log_p (float): Log-probability of `bigram` under this
                `LanguageModel`.
        """
        ### Begin your code

        ### End your code

    def get_query_logp(self, query):
        """Computes the log-probability of `query` under this `LanguageModel`.

        Args:
            query (str): Whitespace-delimited sequence of terms in the query.

        Returns:
            log_p (float): Log-probability assigned to the query under this
                `LanguageModel`.
        """
        ### Begin your code

        ### End your code

In [ ]:
# Make sure your implementation passes the following sanity checks
# Note: Constructing the language model could take 30 seconds or longer
# We suggest using `tqdm` to track progress in your `LanguageModel.__init__` function.
lm = LanguageModel()

print('num. unigrams("{}")'.format(len(lm.unigram_counts))) 
print('num. bigrams("{}")'.format(len(lm.bigram_counts)))
print('num. tokens("{}")'.format(lm.total_num_tokens))


# Test a reasonable query with and without typos (you should try your own)!
query_wo_typo = "sharif university" # write a query without typo
query_w_typo = "sharaf universit"  # write a query with typo

p_wo_typo = math.exp(lm.get_query_logp(query_wo_typo))
p_w_typo = math.exp(lm.get_query_logp(query_w_typo))
print('P("{}") == {}'.format(query_wo_typo, p_wo_typo))
print('P("{}") == {}'.format(query_w_typo, p_w_typo))
if p_wo_typo <= p_w_typo:
    print('Are you sure "{}" should be assigned higher probability than "{}"?'
          .format(query_w_typo, query_wo_typo))
    
print("done!")


<div dir="rtl">

### IV.2.  مدل احتمال ویرایش (Edit Probability Model)

مدل احتمال ویرایش تلاش می‌کند تا مقدار $P(R\mid Q)$ را محاسبه کند. برای یک کوئری نامزد ثابت $Q$، مدل احتمال ویرایش احتمالی را حساب می‌کند که یک کوئری خام احتمالا خطادار $R$ ثبت شده است. در این‌جا فاصله‌ی بین کوئری نامزد $Q$ و ورودی $R$ را با استفاده از [فاصله‌ی Damerau-Levenshtein](https://en.wikipedia.org/wiki/Damerau%E2%80%93Levenshtein_distance) مقداردهی می‌کنیم. در فاصله‌ی Damerau-Levenshtein، ویرایش‌های احتمالی برابرthe possible edits are **درج**، **حذف**، **جایگزینی** و **جابجایی**، که هرکدام شامل تک‌کرکترها به عنوان عملگر هستند. در پایین یک کلاس پایه برای `EditCostModel` را می‌بینید.

In [13]:

class BaseEditProbabilityModel:
    def get_edit_logp(self, edited, original):
        """Gets the log-probability of editing `original` to arrive at `edited`.
        The `original` and `edited` arguments are both single terms that are at
        most one edit apart.
        
        Note: The order of the arguments is chosen so that it reads like an
        assignment expression:
            > edited := EDIT_FUNCTION(original)
        or, alternatively, you can think of it as a (unnormalized) conditional probability:
            > log P(edited | original)

        Args:
            edited (str): Edited term.
            original (str): Original term.

        Returns:
            logp (float): Log-probability of `edited` given `original`
                under this `EditProbabilityModel`.
        """
        raise NotImplementedError  # Force subclass to implement this method

<div dir="rtl">

**نکته مهمی که باید بدانیم این‌است که تابع get_edit_logp با دو ورودی صدا زده می شود که edit-distance آنها یک است.** 
<br>همچنین خروجی نیازی به نرمال شدن ندارد که مجموع احتمالات کانید‌ها یک شود.

</div>

```python
epm = EditProbabilityModelSubclass(...)  # You will define such a subclass later
original = 'user'
edited = 'usre'                      # Edited by transposing 'r' and 'e'
score = epm.get_edit_logp(edited, original)
```

<div dir="rtl">

#### IV.2.1. مدل ویرایش Uniform-Cost

ابتدا *uniform-cost edit model* را پیاده‌سازی می‌کنیم. این مدل محاسبه‌ی احتمال ویرایش را با فرض این که هر ویرایش در فاصله‌ی Damerau-Levenshtein احتمال یکسان دارد، ساده می‌کند. شما باید یک بازه از مقادیر را برای احتمال ویرایش uniform امتحان کنید، اما برای شروع، بازه‌ی 0.01 - 0.10 مناسب است. یک نکته‌ی مهم که باید در ساختن مدل خود به خاطر داشته باشید، این است که کوئری ورودی $R$ کاربر در اکثر مواقع صحیح است (*برای مثال،* $R = Q$). بنابراین باید یک احتمال ثابت بالا برای `edited == original` در نظر گرفته شود؛یک بازه‌ی معقول 0.90 - 0.95 است.

مدل احتمال ویرایشی که در اینجا می‌سازید، زمانی که نامزدهای تصحیح کوئری را رتبه‌بندی می‌کنید استفاده خواهد شد. تولیدکننده‌ی این نامزدها (که در بخش بعدی توضیح داده شده‌ است) یک ویرایش در هر زمان انجام می‌دهد و هر وقت یک ویرایش در یک ترم انجام می‌دهد، مدل احتمال ویرایش را صدا می‌زند و log-probabilities را برای تغییرات multi-edit جمع می‌کند. بنابراین در این قسمت شما تنها باید احتمال `edited` را با توجه به این که این مورد **حداکثر یک ویرایش تا `original` فاصله دارد** حساب کنید. یعنی در اینجا `get_edit_logp` خیلی ساده خواهد بود.

برای پیاده‌سازی مدل ویرایش uniform-cost کلاس زیر را کامل کنید.

In [14]:

class UniformEditProbabilityModel(BaseEditProbabilityModel):
    def __init__(self, edit_prob=0.05):
        """
        Args:
            edit_prob (float): Probability of a single edit occurring, where
                an edit is an insertion, deletion, substitution, or transposition,
                as defined by the Damerau-Levenshtein distance.
        """
        self.edit_prob = edit_prob

    def get_edit_logp(self, edited, original):
        """Gets the log-probability of editing `original` to arrive at `edited`.
        The `original` and `edited` arguments are both single terms that are at
        most one edit apart.
        
        Note: The order of the arguments is chosen so that it reads like an
        assignment expression:
            > edited := EDIT_FUNCTION(original)
        or, alternatively, you can think of it as a (unnormalized) conditional probability:
            > log P(edited | original)

        Args:
            edited (str): Edited term.
            original (str): Original term.

        Returns:
            logp (float): Log-probability of `edited` given `original`
                under this `EditProbabilityModel`.
        """
        ### Begin your code

        ### End your code

In [ ]:
EDIT_PROB = 0.05
epm = UniformEditProbabilityModel(edit_prob=EDIT_PROB)

# Test a basic edit
edited, original = 'usre', 'user'
print(math.isclose(epm.get_edit_logp(edited, original), math.log(EDIT_PROB)))

# Test a non-edit
print( math.isclose(epm.get_edit_logp(original, original), math.log(1. - EDIT_PROB)))


<div dir="rtl">

### IV.3. تولیدکننده‌ی نامزدها 

تولیدکننده‌ی نامزدها یک کوئری خام $R$ که توسط کاربر ثبت شده است را می‌گیرد و برای کوئری مورد نظر $Q$ نامزدها را تولید می‌کند. از آن‌جایی که می‌دانیم بیشتر از 97% خطاهای املایی در یک فاصله ویرایش ۲ از کوئری موردنظر کاربر قرار دارند، پیشنهاد می‌شود اصلاح کوئری‌های موجود در این فاصله را از $R$ در نظر بگیرید. این راه‌حلی است که توسط Peter Norvig در [این مقاله در مورد اصلاح خطاهای املایی](http://norvig.com/spell-correct.html) پیشنهاد شده است. اگرچه استفاده از یک تولیدکننده‌ی بروت فورس خالص که تمام رشته‌های ممکن با فاصله‌ی ۲ از  $R$ را تولید کند راه معقولی نیست، چون برای هر $R$ با طول غیرناچیز، تعداد نامزدها خیلی زیاد خواهد شد. بنابراین ما باید مدل احتمال زبان و ویرایش را برای تعداد زیادی از نامزدها محاسبه کنیم.

#### IV.3.1. تولیدکننده‌ی نامزدها با فضای جستجوی محدود (Candidate Generator with Restricted Search Space)

ما می‌توانیم راه‌حل ساده‌ای را پیش بگیریم که با محدود کردن زیاد فضای جستجو در حین تولید کردن نامزدها قابل مدیریت است. راه‌حل‌های معتبر زیادی برای تولید نامزد بهینه وجود دارد، چند ایده‌ی ساده در زیر آمده:
  - با بررسی *هر ترم، جداگانه* در رشته کوئری $R$ شروع کنید و تمام ویرایش‌های ممکن که با فاصله ۱ از آن‌ ترم وجود دارند را در نظر بگیرید.
  - به یاد داشته باشید که می‌توانید هایفن‌ها (علامت خط تیره) یا اسپیس‌ها را به عنوان عناصر مجموعه‌ی کرکترهای خود در نظر بگیرید. با این کار می‌توانید تعدادی خطای نسبتا رایج را در نظر بگیرید، برای مثال وقتی در میان یک کلمه به اشتباه فاصله می‌افتد یا زمانی که دو ترم در یک کوئری به اشتباه توسط اسپیس جدا می‌شوند و در واقع باید به هم چسبیده باشند.
  - هر وقت برای یک ترم ویرایشی را اعمال می‌کنید، مطمئن شوید که ترم ویرایش شده در دیکشنری ظاهر شود. (به خاطر داشته باشید که فرض کردیم تمام لغات در یک کوئری نامزد معتبر در training corpus ما قابل یافت است).
  - اگر ویرایش‌های ممکن برای چندترم جدا را تولید کردید، محصول کارتزین این ترم‌ها را برای تولید یک کوئری نامزد کامل که شامل ویرایش‌های چندین ترم است را در نظر بگیرید. (اما فراموش نکنید که فاصله ویرایش شما برای کل کوئری جمعا نباید از ۲ بیشتر شود).
  
استراتژی‌های ذکر شده در بالا فقط در حد ایده‌ی اولیه‌ هستند و می‌توانند تا مقدار زیادی گسترش یابند یا تغییر کنند. پیشنهاد می‌کنیم چند گزینه‌ی مختلف را بررسی کنید و در گزارش کتبی خود استراتژی نهایی خود را ذکر کنید و این که چگونه کاربری این استراتژی را به مقدار بهینه رساندید. در نظر داشته باشید که **راه‌حل‌هایی که تمام کوئری‌های نامزد در فاصله‌ی مورد نظر را تولید می‌کنند بسیار کند هستند و نمره‌ی کامل نخواهند داشت.**

In [ ]:

class CandidateGenerator:
    # Alphabet to use for insertion and substitution
    alphabet = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
                'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z',
                '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
                ' ', ',', '.', '-']

    def __init__(self, lm, epm):
        """
        Args:
            lm (LanguageModel): Language model to use for prior probabilities, P(Q).
            epm (EditProbabilityModel): Edit probability model to use for P(R|Q).
        """
        self.lm = lm
        self.epm = epm

    def get_num_oov(self, query):
        """Get the number of out-of-vocabulary (OOV) words in `query`."""
        return sum(1 for w in query.strip().split()
                   if w not in self.lm.unigram_counts)

    def filter_and_yield(self, query, lp):
        if query.strip() and self.get_num_oov(query) == 0:
            yield query, lp

    def get_candidates(self, query):
        """Starts from `query`, and performs EDITS OF DISTANCE <=2 to get new
        candidate queries. To make scoring tractable, only returns/yields
        candidates that satisfy certain criteria (ideas for such criteria are
        described in bullet points above).

        Hint: We suggest you implement a helper function that takes a term and
            generates all possible edits of distance one from that term.
            It should probably only return edits that are in the vocabulary
            (i.e., edits for which `self.get_num_oov(edited) == 0`).

        Args:
            query (str): Starting query.

        Returns:
            Iterable over tuples (cdt, cdt_edit_logp) of candidates and
                their associated edit log-probabilities. Return value could be
                a list or a generator yielding tuples of this form.
        """
        # Yield the unedited query first
        # We provide this line as an example of how to use `self.filter_and_yield`
        yield from self.filter_and_yield(query, self.epm.get_edit_logp(query, query))

        ### Begin your code

        ### End your code

<div dir="rtl">
    می‌توانید تست هایی علاوه بر تست زیر برای ارزیابی اضافه 
    کنید

In [ ]:
cg = CandidateGenerator(lm, epm)
query = 'sharif university'
num_candidates = 0
did_generate_original = False
for candidate, candidate_logp in cg.get_candidates(query):
    num_candidates += 1
    if candidate == query:
        did_generate_original = True
        
    if cg.get_num_oov(query) != 0:
        print("You should not generate queries with out-of-vocab terms ('{}' has OOV terms)".format(candidate))


print(num_candidates)
print(did_generate_original)
### Begin your code

### End your code



<div dir='rtl'>


### IV.4. امتیازدهنده‌ی نامزدها

وظیفه امتیازدهنده‌ی نامزدها این است که شبیه‌ترین کوئری $Q$ را به R پیداکند. این‌کار را با ترکیب مدل زبانی  $P(Q)$ و مدل احتمال ویرایش برای $P(R\mid Q)$ و تولید‌کننده‌ی نامزدها انجام می‌دهد.
$$
    Q^{*} = \arg\max_{Q_{i}} P(Q_{i}\mid R) = \arg\max_{Q_{i}} P(R\mid Q_{i}) P(Q_{i}),
$$
</div>


<div dir="rtl">


#### IV.4.1. امتیازدهنده‌ی نامزدها با وزن‌دهی (Candidate Scorer with Weighting)


بعد از انجام این ترکیب، ما از یک پارامتر برای وزن‌دادن جداگانه به این دومدل استفاده می‌کنیم.
$$
    P(Q\mid R)\propto P(R\mid Q)P(Q)^{\mu}.
$$
با $\mu = 1$ شروع کرده و بعدا مقادیر مختلفی را امتحان کنید تا بهترین نتیجه را بگیرید.
کد پایین را کامل کنید تا مصحح غلط املایی با uniform edit cost model ساخته شود 
</div>


In [19]:

class CandidateScorer:
    """Combines the `LanguageModel`, `EditProbabilityModel`, and
    `CandidateGenerator` to produce the most likely query Q given a raw query R.
    Since the candidate generator already uses the edit probability model, we
    do not need to take the edit probability model as an argument in the constructor.
    """
    def __init__(self, lm, cg, mu=1.):
        """
        Args:
            lm (LanguageModel): Language model for estimating P(Q).
            cg (CandidateGenerator): Candidate generator for generating possible Q.
            mu (float): Weighting factor for the language model (see write-up).
                Remember that our probability computations are done in log-space.
        """
        self.lm = lm
        self.cg = cg
        self.mu = mu

    def get_score(self, query, log_edit_prob):
        """Uses the language model and `log_edit_prob` to compute the final
        score for a candidate `query`. Uses `mu` as weighting exponent for P(Q).

        Args:
            query (str): Candidate query.
            log_edit_prob (float): Log-probability of candidate query given
                original query (i.e., log(P(R|Q), where R is `query`).

        Returns:
            log_p (float): Final score for the query, i.e., the log-probability
                of the query.
        """
        ### Begin your code

        ### End your code

    def correct_spelling(self, r):
        """Corrects spelling of raw query `r` to get the intended query `q`.

        Args:
            r (str): Raw input query from the user.

        Returns:
            q (str): Spell-corrected query. That is, the query that maximizes
                P(R|Q)*P(Q) under the language model and edit probability model,
                restricted to Q's generated by the candidate generator.
        """
        ### Begin your code

        ### End your code

In [ ]:
# Assumes LanguageModel lm was already built above
print('Building edit probability model...')
epm = UniformEditProbabilityModel()
print('Building candidate generator...')
cg = CandidateGenerator(lm, epm)
print('Building candidate scorer model...')
cs = CandidateScorer(lm, cg, mu=1.0)
print('Running spelling corrector...')

# Add your own queries here to test your spelling corrector
queries = [('sharaf university', ' sharif university'),
           ('sharif university', ' sharif university')
           ]
for query, expected in queries:
    corrected = cs.correct_spelling(query)
    print("\t'{}' corrected to '{}'".format(query, corrected))


#### IV.4.2. Dev Set Evaluation (Uniform)

</div>

<div dir="rtl">
حال ما یک تصحیح‌گر غلط املایی اولیه را ساختیم.
برای ارزیابی مدل خود, سل‌های زیر را اجرا کنید (در صورت داشتن دقت بالاتر از ۸۲٪, حداقل ۵ و حداکثر ۱۵درصد نمره امتیازی به شما داده می‌شود(در صورتی که گروه‌های کمی به این دقت رسیدند, نمره امتیازی به صورت رقابتی داده‌خواهد شد)). 
</div>


In [ ]:
def dev_eval(candidate_scorer, verbose=False):
    """Evaluate `candidate_scorer` on the dev set."""
    query_num = 1
    yours_correct = 0
    google_correct = 0
    # Read originals, ground-truths, Google's predictions
    dev_dir = 'MIR2-data/dev_set/'
    with tqdm(total=455, unit=' queries') as pbar, \
            open(os.path.join(dev_dir, 'queries.txt'), 'r') as query_fh, \
            open(os.path.join(dev_dir, 'gold.txt'), 'r') as gold_fh, \
            open(os.path.join(dev_dir, 'google.txt'), 'r') as google_fh:
        while True:
            # Read one line
            query = query_fh.readline().rstrip('\n')
            if not query:
                # Finished all queries
                break
            corrected = candidate_scorer.correct_spelling(query)
            corrected = ' '.join(corrected.split())  # Squash multiple spaces
            gold = gold_fh.readline().rstrip('\n')
            google = google_fh.readline().rstrip('\n')

            # Count whether correct
            if corrected == gold:
                yours_correct += 1
            if google == gold:
                google_correct += 1

            # Print running stats
            yours_accuracy = yours_correct / query_num * 100
            google_accuracy = google_correct / query_num * 100
            if verbose:
                print('QUERY {:03d}'.format(query_num))
                print('---------')
                print('(original):      {}'.format(query))
                print('(corrected):     {}'.format(corrected))
                print('(google):        {}'.format(google))
                print('(gold):          {}'.format(gold))
                print('Google accuracy: {}/{} ({:5.2f}%)\n'
                      .format(google_correct, query_num, google_accuracy))
                print('Your accuracy:   {}/{} ({:5.2f}%)'
                      .format(yours_correct, query_num, yours_accuracy))
            
            pbar.set_postfix(google='{:5.2f}%'.format(google_accuracy),
                             yours='{:5.2f}%'.format(yours_accuracy))
            pbar.update()
            query_num += 1

In [ ]:
# Set verbose=True for debugging output
dev_eval(cs, verbose=False)

<a id='empirical'></a>
<div dir="rtl">


##  تصحیح املا با هزینه‌های ویرایش تجربی (امتیازی ۲۰ درصد)

### V.1. مدل احتمال ویرایش بهبودیافته (Improved Edit Probability Model)


</div>
<div dir='rtl'>
حال که تصحیح‌کننده غلط املایی ما با یک مدل احتمالاتی ویرایش اولیه(edit probability model) به درستی کار می‌کند, برای ویرایش احتمالات به سمت یک رویکرد کاربردی‌تر می‌رویم. برای این بخش و learn کردن این احتمال ویرایش‌ها ما از داده‌ی خطای تجربی که در  
`data/training_set/edit1s.txt` قراردارد استفاده می‌کنیم

</div>



<div dir='rtl'>

#### V.1.1. هزینه‌ها‌ی ویرایش تجربی (Empirical Edit Costs) 
</div>

<div dir='rtl'>



یک لیست از جفت کوئری‌هایی که فاصله‌یشان از یکدیگر دقیقا ۱ است به شما داده شده است. در قدم اول یک الگوریتم ساده برای تعیین کردن این که کدام ویرایش بخصوص میان دو کوئری در هر جفت وجود دارد پیاده‌سازی کنید. با جمع کردن شمارش تمام ویرایش‌های این‌چنینی در تمام کوئری‌ها احتمال هر ویرایش قابل محاسبه خواهد بود. به عنوان مثال، اگر خواستید تعیین کنید که احتمال جایگزین شدن اشتباهی حرف 'e' با حرف 'a' در یک کوئری چقدر است، باید مقدار زیر را محاسبه کنید:
$$
    P(\texttt{sub}[a, e]) = \frac{\texttt{count}(\texttt{sub}[a, e])}{\texttt{count}(e)}.
$$
با توجه به این که احتمال عملیات‌های درج و حذف به حروف قبل از حرفی که روی آن عملی انجام می‌شود مشروطند، پس باید راه‌حل مناسبی برای حذف و درج در ابتدای یک لغت ارائه دهید. در نهایت برای رفع مشکل پراکندگی داده‌ها در فایل training روش هموارسازی Laplace add-one را برای احتمالات ویرایش پیاده‌سازی کنید.

In [16]:

class Edit:
    """Represents a single edit in Damerau-Levenshtein distance.
    We use this class to count occurrences of different edits in the training data.
    """
    INSERTION = 1
    DELETION = 2
    TRANSPOSITION = 3
    SUBSTITUTION = 4

    def __init__(self, edit_type, c1=None, c2=None):
        """
        Members:
            edit_type (int): One of Edit.{NO_EDIT,INSERTION,DELETION,
                TRANSPOSITION,SUBSTITUTION}.
            c1 (str): First (in original) char involved in the edit.
            c2 (str): Second (in original) char involved in the edit.
        """
        self.edit_type = edit_type
        self.c1 = c1
        self.c2 = c2


class EmpiricalEditProbabilityModel(BaseEditProbabilityModel):

    START_CHAR = ''      # Used to indicate start-of-query
    NO_EDIT_PROB = 0.92  # Hyperparameter for probability assigned to no-edit

    def __init__(self, training_set_path='MIR2-data/training_set/edit1s.txt'):
        """Builds the necessary data structures to compute log-probabilities of
        distance-1 edits in constant time. In particular, counts the unigrams
        (single characters), bigrams (of 2 characters), alphabet size, and
        edit count for insertions, deletions, substitutions, and transpositions.

        Hint: Use the `Edit` class above. It may be easier to write the `get_edit`
        function first, since you should call that function here.

        Note: We suggest using tqdm with the size of the training set (819722) to track
        the initializers progress when parsing the training set file.

        Args:
            training_set_path (str): Path to training set of empirical error data.
        """
        # Your code needs to initialize all four of these data structures
        self.unigram_counts = Counter()  # Maps chars c1 -> count(c1)
        self.bigram_counts = Counter()   # Maps tuples (c1, c2) -> count((c1, c2))
        self.alphabet_size = 0           # Counts all possible characters

        # Maps edit-types -> dict mapping tuples (c1, c2) -> count(edit[c1, c2])
        # Example usage: 
        #   > e = Edit(Edit.SUBSTITUTION, 'a', 'b')
        #   > edit_count = self.edit_counts[e.edit_type][(e.c1, e.c2)]
        self.edit_counts = {edit_type: Counter()
                            for edit_type in (Edit.INSERTION, Edit.DELETION,
                                              Edit.SUBSTITUTION, Edit.TRANSPOSITION)}

        with open(training_set_path, 'r') as training_set:
            for example in tqdm(training_set, total=819722):
                edited, original = example.strip().split('\t')

                ### Begin your code

                ### End your code

    def get_edit(self, edited, original):
        """Gets an `Edit` object describing the type of edit performed on `original`
        to produce `edited`.

        Note: Only edits with an edit distance of at most 1 are valid inputs.

        Args:
            edited (str): Raw query, which contains exactly one edit from `original`.
            original (str): True query. Want to find the edit which turns this into `edited`.

        Returns:
            edit (Edit): `Edit` object representing the edit to apply to `original` to get `edited`.
                If `edited == original`, returns None.
        """
        ### Begin your code

        ### End your code

    def get_edit_logp(self, edited, original):
        """Gets the log-probability of editing `original` to arrive at `edited`.
        The `original` and `edited` arguments are both single terms that are at
        most one edit apart.
        
        Note: The order of the arguments is chosen so that it reads like an
        assignment expression:
            > edited := EDIT_FUNCTION(original)
        or, alternatively, you can think of it as a (unnormalized) conditional probability:
            > log P(edited | original)

        Args:
            edited (str): Edited term.
            original (str): Original term.

        Returns:
            logp (float): Log-probability of `edited` given `original`
                under this `EditProbabilityModel`.
        """
        ### Begin your code

        ### End your code

In [ ]:
# Build spelling corrector for evaluation on the dev set
lm = LanguageModel()
epm = EmpiricalEditProbabilityModel()
cg = CandidateGenerator(lm, epm)
cs = CandidateScorer(lm, cg, mu=1.0)

<div dir='rtl'>
     . در صورت داشتن دقت بالاتر از ۸۱٪, ۵ تا ۱۵درصد نمره امتیازی به شما داده می‌شود (در صورتی که گروه‌های کمی به این دقت رسیدند, نمره امتیازی به صورت رقابتی داده‌خواهد شد)
    </div

In [ ]:
# Set verbose=True for debugging output
dev_eval(cs, verbose=False)

<a id='written'></a>
<div dir="rtl">


##  گزارش مکتوب



</div>

<div dir="rtl">


در این بخش انتظار می‌رود که شما گزارشی از کارهای انجام شده در هر تسک - نتایج گرفته شده و ایده‌هایی که استفاده کردید و نیز تغییرات داده شده در کد(در صورت وجود) بنویسید. این بخش ۱۵درصد نمره شما را خواهد داشت


</div>